In [18]:
!pip install gdown -U
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import os
os.environ["WANDB_DISABLED"]="true"

In [20]:
!mkdir -p results_promise_docs_bert

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
!gdown '1zLzqY2h10TNiq9NDKl0cJBTWSfHv014Y'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1zLzqY2h10TNiq9NDKl0cJBTWSfHv014Y
To: /kaggle/working/NFR_PROMISE.csv
100%|██████████████████████████████████████| 81.8k/81.8k [00:00<00:00, 82.1MB/s]


Aqui empiezo el código igual al de colab

In [22]:
from datasets import load_dataset

In [23]:
FILE_REQ='/kaggle/working/NFR_PROMISE.csv'
data_req = load_dataset('csv', data_files=FILE_REQ, sep=';')
print(data_req['train']['NFR'][21])

def create_labels(row):
  return {'labels': 'F' if row['NFR'] == 'F' else 'NF'}

data_req = data_req.map(create_labels)
data_req = data_req.class_encode_column('labels')
print(set(data_req['train']['labels']))
data_req = data_req['train']
data_req.features

F
{0, 1}


{'Num': Value(dtype='int64', id=None),
 'Doc': Value(dtype='int64', id=None),
 'Requisito': Value(dtype='string', id=None),
 'NFR': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['F', 'NF'], id=None)}

In [24]:
from transformers import AutoTokenizer
#BERT: model_name = 'google-bert/bert-base-cased'
#DEBERTA: model_name = 'microsoft/deberta-v3-base'
#ROBERTA: model_name = 'FacebookAI/roberta-base'
model_name = 'google-bert/bert-base-cased' # modificar con el modelo que queramos cada vez

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
def encode_text(row, tokenizer):
  row_encode = tokenizer(row['Requisito'], truncation=True)
  return row_encode


encode_data = data_req.map(encode_text, fn_kwargs={'tokenizer':tokenizer})
encode_data

from sklearn.model_selection import StratifiedKFold
import numpy as np

folds = StratifiedKFold(n_splits=10, shuffle=True)

splits = folds.split(np.zeros(data_req.num_rows), encode_data["labels"])
test_sets = []
train_val_sets = []
i=1
for train_val_idxs, test_idxs in splits:
  print (i)
  print(test_idxs)
  print('*******')
  data_test = encode_data.select(test_idxs)
  data_train = encode_data.select(train_val_idxs)
  data_train_val = data_train.train_test_split(train_size=0.9, stratify_by_column='labels')
  #data_train_inner, data_val_inner = data_train.train_test_split(train_size=0.9)
  test_sets.append(data_test)
  train_val_sets.append(data_train_val)
  i = i + 1

print(test_sets[0])
print(train_val_sets[0]['test'])

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

1
[  1   2  22  36  40  50  68  73  79  88 117 118 125 136 142 157 160 175
 177 181 199 216 220 226 227 238 243 248 268 274 277 298 322 323 325 333
 358 360 369 384 389 397 414 415 416 440 449 456 465 472 483 486 492 502
 504 509 538 540 568 572 578 600 618]
*******
2
[  4  31  33  42  52  56  62  63  67  78  80  94  96 107 111 124 127 129
 140 158 169 182 187 235 254 258 263 264 290 297 299 302 321 336 338 344
 364 368 406 408 412 426 434 470 471 475 479 499 503 513 523 527 542 544
 561 567 569 575 589 590 596 597 620]
*******
3
[  6   8   9  14  39  51  75  77  87  90  93 100 131 141 143 144 148 154
 155 167 178 194 196 218 222 240 242 265 267 279 280 289 292 295 310 348
 359 395 398 400 410 429 435 443 450 468 477 488 489 508 510 516 520 528
 539 566 570 574 580 612 613 621 624]
*******
4
[  7  11  13  17  19  29  45  55  76  82  91 114 126 161 162 174 191 198
 207 224 232 236 252 253 259 272 275 287 293 300 301 307 308 320 337 339
 349 353 357 363 367 379 386 390 394 403 418 424 43

In [27]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import evaluate
import numpy as np

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels)

# load metric
metric_name = 'f1'
metric = evaluate.load(metric_name)

Si uso dos GPUs poner batch size 16, porque van 16 a cada una = 32

In [30]:
import glob
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report

lr_values =[2e-5, 5e-5]

for j in range(len(train_val_sets)):
    best_lr = -1
    best_f1 = -1

    # para escribir los resultados
    with open(f'results_promise_docs_bert/res_bert_{j}.txt', 'w') as f:
    # NOTA: si no va, deshacer el tab extra del siguiente bucle for
      for i, lr in enumerate(lr_values):
        print("================================================")
        print("j = " + str(j) + " // lr = " + str(lr))
        print("================================================")
      # argumentos para el entrenamiento
        training_args = TrainingArguments(
            #output_dir="my_checkpoint"+str(j),
            output_dir=f"my_checkpoint_{i}",
            overwrite_output_dir = True,
            num_train_epochs=10,
            #para probar tanto promise como nuestro coger 10 epochs
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit = 1,
            load_best_model_at_end=True,
            logging_strategy='epoch',
            optim="adamw_torch",
            per_device_train_batch_size=32,
            #lo dejaremos en 32: per_device_eval_batch_size=32,
            learning_rate=lr,
            #probar tanto 2e-5 como 5e-5
            weight_decay=0.01,
        )

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        # declarar el "entrenador"
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_val_sets[j]['train'],
            eval_dataset=train_val_sets[j]['test'],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # realizar el entrenamiento
        trainer.train()

        out_pred = trainer.predict(train_val_sets[j]['test'])
        if out_pred.metrics['test_f1'] > best_f1:
          best_f1 = out_pred.metrics['test_f1']
          best_lr = i
      # cargar el mejor modelo y resultados para test
      best_dir = glob.glob(f"my_checkpoint_{best_lr}/checkpoint*")
      print(best_dir)
      model_best = AutoModelForSequenceClassification.from_pretrained(best_dir[0])
      trainer_best = Trainer(
            model=model_best,
            args=None,
            train_dataset=None,
            eval_dataset=None,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

      test_data = test_sets[j]
      outputs_pred = trainer_best.predict(test_dataset=test_data)

      # model predictions
      predictions = np.argmax(outputs_pred.predictions, axis=1)

      #d_res = classification_report(test_data['labels'], predictions, digits=3, return_dict = True)
      d_res = classification_report(test_data['labels'], predictions, digits=3, output_dict=True)
      print(d_res)
      f.write("================== Best lr ================\n")
      f.write(str(best_lr))
      f.write("\n")
      f.write( "================== Resultados ================\n")
      f.write(str(d_res))

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


j = 0 // lr = 2e-05


Epoch,Training Loss,Validation Loss,F1
1,0.675500,0.653079,0.747253
2,0.564800,0.285978,0.914286
3,0.256500,0.146408,0.953846
4,0.157900,0.138448,0.927536
5,0.109200,0.182466,0.941176
6,0.066400,0.119527,0.955224
7,0.058900,0.144745,0.970588
8,0.033800,0.141236,0.955224
9,0.027100,0.173096,0.955224
10,0.028200,0.178862,0.955224


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


j = 0 // lr = 5e-05


...

j = 9 // lr = 5e-05


Epoch,Training Loss,Validation Loss,F1
1,0.617200,0.463583,0.842105
2,0.339900,0.241925,0.925373
3,0.169800,0.127484,0.971429
4,0.100000,0.251119,0.923077
5,0.047700,0.122435,0.955224
6,0.047200,0.242869,0.957746
7,0.016600,0.248334,0.956522
8,0.007900,0.260686,0.971429
9,0.008400,0.380050,0.941176
10,0.003600,0.387468,0.941176


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


['my_checkpoint_1/checkpoint-80']


{'0': {'precision': 0.9583333333333334, 'recall': 0.92, 'f1-score': 0.9387755102040817, 'support': 25}, '1': {'precision': 0.9473684210526315, 'recall': 0.972972972972973, 'f1-score': 0.9599999999999999, 'support': 37}, 'accuracy': 0.9516129032258065, 'macro avg': {'precision': 0.9528508771929824, 'recall': 0.9464864864864866, 'f1-score': 0.9493877551020408, 'support': 62}, 'weighted avg': {'precision': 0.9517897566496888, 'recall': 0.9516129032258065, 'f1-score': 0.9514417379855167, 'support': 62}}


Aquí acaba el código de colab. 
Falta hacer el zip para poder descargar los resultados (y opcionalmente usar el bot para que me avise del final de la ejecución)

In [31]:
import requests

# Definir la URL base y los parámetros
url = "https://api.callmebot.com/whatsapp.php"
params = {
    "phone": "+34653508040",
    "apikey": "1512070",
    "text": "🤖¡Ejecución terminada! 🤖\nVe a echarle un vistazo ☝🤓"
}

# Enviar la solicitud GET
response = requests.get(url, params=params)

# Imprimir el estado de la respuesta
print(response.status_code)
print(response.text)

200
<p>Message to: +34653508040<p>Text to send: 🤖¡Ejecución terminada! 🤖%0AVe a echarle un vistazo ☝🤓<p><b>Message queued.</b> You will receive it in a few seconds.


In [32]:
!zip -r results_promise_docs_bert.zip results_promise_docs_bert
from IPython.display import FileLink 
FileLink(r'results_promise_docs_bert.zip')

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: results_promise_stratified_roberta/ (stored 0%)
  adding: results_promise_stratified_roberta/res_roberta_7.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_8.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_0.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_9.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_3.txt (deflated 61%)
  adding: results_promise_stratified_roberta/res_roberta_1.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_2.txt (deflated 62%)
  adding: results_promise_stratified_roberta/res_roberta_6.txt (deflated 58%)
  adding: results_promise_stratified_roberta/res_roberta_4.txt (deflated 60%)
  adding: results_promise_stratified_roberta/res_roberta_5.txt (deflated 60%)


/kaggle/working/results_promise_stratified_roberta.zip